In [ ]:
import numpy as np
import scipy.signal as signal
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

**Defining the AHP function**

In [ ]:
import numpy as np

def calculate_ahp_weights(criteria_matrix):
    num_criteria = criteria_matrix.shape[0]
    eigvals, eigenvectors = np.linalg.eig(criteria_matrix)

    # Find the index of the maximum eigenvalue
    max_eigenvalue_index = np.argmax(eigvals)

    # Get the corresponding eigenvector
    max_eigenvalue_vector = eigenvectors[:, max_eigenvalue_index]

    # Normalize the eigenvector
    sum_eigenvector = np.sum(max_eigenvalue_vector)

    if sum_eigenvector != 0:
        normalized_eigenvector = max_eigenvalue_vector / sum_eigenvector
    else:
        normalized_eigenvector = max_eigenvalue_vector

    return normalized_eigenvector

**Define the Markov Function**

In [ ]:
def ca_markov_model(prev_state, parameters, weights):
    kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])  # Define the kernel for convolution
    next_state = signal.convolve2d(prev_state, kernel, mode='same', boundary='wrap')  # Apply convolution
    urban_expansion=parameters[0]*weights[0]+parameters[1]*weights[1]+parameters[2]*weights[2]+parameters[3]*weights[3]+parameters[4]*weights[4]+parameters[5]*weights[5]
    # Add urban expansion to the next state
    next_state += ((urban_expansion))
    # Perform thresholding to determine urban areas
    threshold = 0.95  # Define the threshold value
    next_state[next_state >= threshold] = 1
    next_state[next_state < threshold] = 0

    return next_state

**Load and Preprocess data**

In [ ]:
from PIL import Image

builtup_2005 = Image.open('/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif')
builtup_2011 = Image.open('/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/11_12_builtup_some.tif')
builtup_2015 = Image.open('/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/15_16_builtup.tif')
builtup_2020 = Image.open('/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/20_21_builtup.tif')

target_size =(2719,1283)
builtup_2020=builtup_2020.resize(target_size)

parameters_2023_paths= ['/content/drive/MyDrive/PS-1 Urban/All new files/Accessibilty.tif','/content/drive/MyDrive/PS-1 Urban/All new files/Pop_den.tif','/content/drive/MyDrive/PS-1 Urban/All new files/Inverted_Elevation.tif','/content/drive/MyDrive/PS-1 Urban/All new files/Inverted_Floods.tif','/content/drive/MyDrive/PS-1 Urban/All new files/Inverted_Slope.tif','/content/drive/MyDrive/PS-1 Urban/All new files/Friction.tif']
parameters_2023=[]
for path in parameters_2023_paths:
  image= Image.open(path)
  image= image.resize(target_size)
  parameters_2023.append(image)

In [ ]:
# Converting data into numpy arrays
builtup_2005_array= np.array(builtup_2005)
builtup_2011_array= np.array(builtup_2011)
builtup_2015_array= np.array(builtup_2015)
builtup_2020_array= np.array(builtup_2020)

parameters_2023_array=[]
for image in parameters_2023:
  array=np.array(image)
  parameters_2023_array.append(array)

In [ ]:
print(builtup_2005_array.shape)
print(builtup_2011_array.shape)
print(builtup_2015_array.shape)
print(builtup_2020_array.shape)

for i in [0,1,2,3,4,5]:
  print(parameters_2023_array[i].shape)

(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)


In [ ]:
import numpy as np
from skimage.util import img_as_float

_2005_float=img_as_float(builtup_2005_array)
_2011_float=img_as_float(builtup_2011_array)
_2015_float=img_as_float(builtup_2015_array)
_2020_float=img_as_float(builtup_2020_array)

parameters_2023_float=[]
for i in [0,1,2,3,4,5]:
  parameters_2023_float.append(img_as_float(parameters_2023_array[i]))

nan_mask_2005=np.isnan(_2005_float)
nan_mask_2011=np.isnan(_2011_float)
nan_mask_2015=np.isnan(_2015_float)
nan_mask_2020=np.isnan(_2020_float)

nan_mask_parameters=[]
for i in [0,1,2,3,4,5]:
  nan_mask_parameters.append(np.isnan(parameters_2023_float[i]))

_2005_without_nan=np.where(nan_mask_2005, -100, _2005_float)
_2011_without_nan=np.where(nan_mask_2011, -100, _2011_float)
_2015_without_nan=np.where(nan_mask_2015, -100, _2015_float)
_2020_without_nan=np.where(nan_mask_2020, -100, _2020_float)

parameters_without_nan=[]
for i in [0,1,2,3,4,5]:
  parameters_without_nan.append(np.where(nan_mask_parameters[i], -100, parameters_2023_float[i]))

In [ ]:
builtup_2005_array=_2005_without_nan
builtup_2011_array=_2011_without_nan
builtup_2015_array=_2015_without_nan
builtup_2020_array=_2020_without_nan

parameters_2023_array=parameters_without_nan

In [ ]:
print(builtup_2005_array.shape)
print(builtup_2011_array.shape)
print(builtup_2015_array.shape)
print(builtup_2020_array.shape)

for i in [0,1,2,3,4,5]:
  print(parameters_2023_array[i].shape)
print(np.array(parameters_2023_array).shape)

(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(1283, 2719)
(6, 1283, 2719)


**Calculate the AHP weights**

In [ ]:
criteria_matrix = np.ones((6, 6))
criteria_matrix[0, 0] = 1
criteria_matrix[0, 1] = 7
criteria_matrix[0, 2] = 3
criteria_matrix[0, 3] = 7
criteria_matrix[0, 4] = 6
criteria_matrix[0, 5] = 6
criteria_matrix[1, 0] = 1/7
criteria_matrix[1, 1] = 1
criteria_matrix[1, 2] = 1/5
criteria_matrix[1, 3] = 2
criteria_matrix[1, 4] = 1/2
criteria_matrix[1, 5] = 1/2
criteria_matrix[2, 0] = 1/3
criteria_matrix[2, 1] = 5
criteria_matrix[2, 2] = 1
criteria_matrix[2, 3] = 6
criteria_matrix[2, 4] = 3
criteria_matrix[2, 5] = 3
criteria_matrix[3, 0] = 1/7
criteria_matrix[3, 1] = 1/2
criteria_matrix[3, 2] = 1/6
criteria_matrix[3, 3] = 1
criteria_matrix[3, 4] = 1/2
criteria_matrix[3, 5] = 1/2
criteria_matrix[4, 0] = 1/6
criteria_matrix[4, 1] = 2
criteria_matrix[4, 2] = 1/3
criteria_matrix[4, 3] = 2
criteria_matrix[4, 4] = 1
criteria_matrix[4, 5] = 2
criteria_matrix[5, 0] = 1/6
criteria_matrix[5, 1] = 2
criteria_matrix[5, 2] = 1/3
criteria_matrix[5, 3] = 2
criteria_matrix[5, 4] = 1/2
criteria_matrix[5, 5] = 1

#Calculating the weights
weights= calculate_ahp_weights(criteria_matrix)

print(weights)

[0.48103461+0.j 0.05658601+0.j 0.23956212+0.j 0.04366571+0.j
 0.10001182+0.j 0.07913973+0.j]


In [ ]:
import numpy as np

def calculate_consistency_index(matrix):
    n = matrix.shape[0]
    eigenvalues, _ = np.linalg.eig(matrix)
    lambda_max = np.max(eigenvalues.real)
    consistency_index = (lambda_max - n) / (n - 1)
    return consistency_index

def calculate_random_index(n):
    random_index_table = [0, 0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49]
    if n < 1 or n > 10:
        raise ValueError("Number of criteria out of range (1-10)")
    return random_index_table[n - 1]

def calculate_consistency_ratio(ci, ri):
    consistency_ratio = ci / ri
    return consistency_ratio

ci = calculate_consistency_index(criteria_matrix)
ri = calculate_random_index(criteria_matrix.shape[0])
cr = calculate_consistency_ratio(ci, ri)

print(f"Consistency Index (CI): {ci}")
print(f"Random Index (RI): {ri}")
print(f"Consistency Ratio (CR): {cr}")


Consistency Index (CI): 0.03749613775072511
Random Index (RI): 1.24
Consistency Ratio (CR): 0.0302388207667138


In [ ]:
print(type(weights))
print(weights.shape)
sum=0
for i in [0,1,2,3,4,5]:
  sum=sum+weights[i]

print(sum)

<class 'numpy.ndarray'>
(6,)
(1+0j)


In [ ]:
weights=np.real(weights)

print(weights)

[0.48103461 0.05658601 0.23956212 0.04366571 0.10001182 0.07913973]


In [ ]:
parameters_2023_array = np.array(parameters_2023_array)
parameters_2023_array.shape

(6, 1283, 2719)

In [ ]:
builtup_2020_array[builtup_2020_array>=0.99]=1
builtup_2020_array[builtup_2020_array<0.99]=0

**Training data**

In [ ]:
# Train the model (predict 2011 from 2005)
from sklearn.metrics import confusion_matrix
initial_state = builtup_2005_array
for i in range(1, 7):
    predicted_state_2011_05 = ca_markov_model(initial_state,parameters_2023_array,weights)
    initial_state=predicted_state_2011_05

a = confusion_matrix(builtup_2011_array.flatten(),predicted_state_2011_05.flatten())
print(f"Confusion Matrix: \n {a}")
print(f"Validation Accuracy: {((a[1][1]+a[2][2])/(a[1][1]+a[2][2]+a[2][1]+a[1][2]))*100}")

Confusion Matrix: 
 [[      0 1549209       0]
 [      0 1547571  137592]
 [      0   27872  226233]]
Validation Accuracy: 91.46770843431645


In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2011_pred_from_2005_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2011_05)
output_band.FlushCache()

# Close the output image
output_image = None

In [ ]:
# Train the model (predict 2015 from 2005)
from sklearn.metrics import confusion_matrix
initial_state = builtup_2005_array
for i in range(1, 11):
    predicted_state_2015_05 = ca_markov_model(initial_state,parameters_2023_array,weights)
    initial_state=predicted_state_2015_05

a = confusion_matrix(builtup_2015_array.flatten(),predicted_state_2015_05.flatten())
print(f"Confusion Matrix: \n {a}")
print(f"Validation Accuracy: {((a[1][1]+a[2][2])/(a[1][1]+a[2][2]+a[2][1]+a[1][2]))*100}")

Confusion Matrix: 
 [[      0 1549209       0]
 [      0 1487425  165937]
 [      0   39850  246056]]
Validation Accuracy: 89.38841872294083


In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2015_pred_from_2005_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2015_05)
output_band.FlushCache()

# Close the output image
output_image = None

In [ ]:
# Train the model (predict 2020 from 2005)
from sklearn.metrics import confusion_matrix
initial_state = builtup_2005_array
for i in range(1, 16):
    predicted_state_2020_05 = ca_markov_model(initial_state,parameters_2023_array,weights)
    initial_state=predicted_state_2020_05

a = confusion_matrix(builtup_2020_array.flatten(),predicted_state_2020_05.flatten())
print(f"Confusion Matrix: \n {a}")
print(f"Validation Accuracy: {((a[1][1]+a[0][0])/(a[1][1]+a[0][0]+a[0][1]+a[1][0]))*100}")

Confusion Matrix: 
 [[2997642  314662]
 [  27367  148806]]
Validation Accuracy: 90.19546352176036


In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2020_pred_from_2005_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2020_05)
output_band.FlushCache()

# Close the output image
output_image = None

In [ ]:
# Train the model (predict 2015 from 2011)
from sklearn.metrics import confusion_matrix
initial_state = builtup_2011_array
for i in range(1, 5):
    predicted_state_2015_11 = ca_markov_model(initial_state,parameters_2023_array,weights)
    initial_state=predicted_state_2015_11

a = confusion_matrix(builtup_2015_array.flatten(),predicted_state_2015_11.flatten())
print(f"Confusion Matrix: \n {a}")
print(f"Validation Accuracy: {((a[1][1]+a[2][2])/(a[1][1]+a[2][2]+a[2][1]+a[1][2]))*100}")

Confusion Matrix: 
 [[      0 1549209       0]
 [      0 1546757  106605]
 [      0   30730  255176]]
Validation Accuracy: 92.91820418838448


In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2015_pred_from_2011_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2015_11)
output_band.FlushCache()

# Close the output image
output_image = None

In [ ]:
# Train the model (predict 2020 from 2011)
from sklearn.metrics import confusion_matrix
initial_state = builtup_2011_array
for i in range(1, 10):
    predicted_state_2020_11 = ca_markov_model(initial_state,parameters_2023_array,weights)
    initial_state=predicted_state_2020_11

a = confusion_matrix(builtup_2020_array.flatten(),predicted_state_2020_11.flatten())
print(f"Confusion Matrix: \n {a}")
print(f"Validation Accuracy: {((a[1][1]+a[0][0])/(a[1][1]+a[0][0]+a[0][1]+a[1][0]))*100}")

Confusion Matrix: 
 [[3006538  305766]
 [  22453  153720]]
Validation Accuracy: 90.59133828315336


In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2020_pred_from_2011_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2020_11)
output_band.FlushCache()

# Close the output image
output_image = None

In [ ]:
# Train the model (predict 2020 from 2015)
from sklearn.metrics import confusion_matrix
initial_state = builtup_2015_array
for i in range(1, 6):
    predicted_state_2020_15 = ca_markov_model(initial_state,parameters_2023_array,weights)
    initial_state=predicted_state_2020_15

a = confusion_matrix(builtup_2020_array.flatten(),predicted_state_2020_15.flatten())
print(f"Confusion Matrix: \n {a}")
print(f"Validation Accuracy: {((a[1][1]+a[0][0])/(a[1][1]+a[0][0]+a[0][1]+a[1][0]))*100}")

Confusion Matrix: 
 [[3051673  260631]
 [  22286  153887]]
Validation Accuracy: 91.88995656270632


In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2020_pred_from_2015_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2020_11)
output_band.FlushCache()

# Close the output image
output_image = None

**Predicting 2031**

In [ ]:
# Predict 2031 urbanization using 2015 data

initial_state_2015= builtup_2015_array
for i in range(1,17):
  predicted_state_2031_15=ca_markov_model(initial_state_2015,parameters_2023_array,weights)
  initial_state_2015=predicted_state_2031_15

In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2031_pred_from_2015_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2031_15)
output_band.FlushCache()

# Close the output image
output_image = None

In [ ]:
# Predict 2031 urbanization using 2020 data

initial_state_2020= builtup_2020_array
for i in range(1,12):
  predicted_state_2031_20=ca_markov_model(initial_state_2020,parameters_2023_array,weights)
  initial_state_2020=predicted_state_2031_20

In [ ]:
import numpy as np
from osgeo import gdal

# Load the input image
input_image = gdal.Open("/content/drive/MyDrive/PS-1 Urban/All new files/New Builtup/05_06_builtup.tif")
input_array = input_image.ReadAsArray()

# Create a new georeferenced output image based on the input image
driver = gdal.GetDriverByName("GTiff")
output_image = driver.Create("/content/drive/MyDrive/PS-1 Urban/CA_Markov_Predictions/2031_pred_from_2020_2.tif", input_image.RasterXSize, input_image.RasterYSize, 1, gdal.GDT_Float32)
output_image.SetGeoTransform(input_image.GetGeoTransform())
output_image.SetProjection(input_image.GetProjection())

# Write the predicted state to the output image
output_band = output_image.GetRasterBand(1)
output_band.WriteArray(predicted_state_2031_20)
output_band.FlushCache()

# Close the output image
output_image = None

**Here you can see the maps of different iterations**

[maps](https://drive.google.com/drive/folders/1Z59Fp41CzJxAHOC51hCt_mJanFhPUZFh?usp=drive_link)

** Here you can look into .tiff images also of the predictions **
[CA MARKOV PREDICTIONS](https://drive.google.com/drive/folders/1-YFu8jbBrzx0lFNpzg78rb4eV1EsxI4u?usp=drive_link)